In [149]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


# Libraries

In [150]:
# torch stuff
import torch
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

# needed for model
from math import log2
from tqdm import tqdm
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import torchvision
from torchvision.utils import save_image

# GAN Variables

In [151]:
datapath= "/home/dsu/Desktop/Projects/FinalProject/cat_imgs"

gen_save_path = "gen_gan.pth"
dis_save_path = "dis_gan.pth"

if(torch.cuda.is_available()):
    device = "cuda"
else:
    print("Warning! Using CPU!")
    device = "cpu"

# If load_model is False, model will train. 
# If load_model is true, no training will happen, only display generated images
save_model = True
load_model = False

lr = 1e-3
batch_size = 64

noise_dim = 100
img_size = 64
channels = 3

relu_slope = 0.2
dropout = 0.3
momentum = 0.1 # 0.9?

grad_pen = True

total_epochs = 250

# Generator

In [152]:
class Generator(nn.Module):
    def __init__(self, noise_dim, in_channels, img_channels=3):
        super(Generator, self).__init__()
        
        self.conv2d_1 = nn.ConvTranspose2d(noise_dim, 512, kernel_size=4, stride=1, padding=0, bias = False)
        self.batch_norm1 = nn.BatchNorm2d(512, momentum=momentum)
        self.relu1 = nn.LeakyReLU(relu_slope) 

        self.conv2d_2 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias = False)
        self.batch_norm2 = nn.BatchNorm2d(256, momentum=momentum)
        self.relu2 = nn.LeakyReLU(relu_slope) 

        self.conv2d_3 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias = False)
        self.batch_norm3 = nn.BatchNorm2d(128, momentum=momentum)
        self.relu3 = nn.LeakyReLU(relu_slope) 

        self.conv2d_4 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias = False)
        self.batch_norm4 = nn.BatchNorm2d(64, momentum=momentum)
        self.relu4 = nn.LeakyReLU(relu_slope) 

        # 5th block? (if so, use 1024 to start)
        #self.conv2d_5 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias = False)
        #self.batch_norm5 = nn.BatchNorm2d(64, momentum=momentum)
        #self.relu5 = nn.LeakyReLU(relu_slope) 

        self.conv2d_final = nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1, bias=False)
        self.tanh = nn.Tanh()
        # tanh?        




    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.batch_norm1(x)
        x = self.relu1(x)

        x = self.conv2d_2(x)
        x = self.batch_norm2(x)
        x = self.relu2(x)

        x = self.conv2d_3(x)
        x = self.batch_norm3(x)
        x = self.relu3(x)

        x = self.conv2d_4(x)
        x = self.batch_norm4(x)
        x = self.relu4(x)

        #x = self.conv2d_5(x)
        #x = self.batch_norm5(x)
        #x = self.relu5(x)

        x = self.conv2d_final(x)
        x = self.tanh(x)

        return x

# Discriminator

In [153]:
class Discriminator(nn.Module):
    def __init__(self, in_channels, img_channels=3):
        super(Discriminator, self).__init__()


        self.conv2d_1 = nn.Conv2d(img_channels, 64, kernel_size=4, stride=2, padding=1, bias=False)
        # no batch norm because first layer
        self.relu1 = nn.LeakyReLU(relu_slope)
        self.dropout1 = nn.Dropout(dropout)

        self.conv2d_2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(128, momentum=momentum)
        self.relu2 = nn.LeakyReLU(relu_slope) 
        self.dropout2 = nn.Dropout(dropout)

        self.conv2d_3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch_norm3 = nn.BatchNorm2d(256, momentum=momentum)
        self.relu3 = nn.LeakyReLU(relu_slope) 
        self.dropout3 = nn.Dropout(dropout)

        self.conv2d_4 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch_norm4 = nn.BatchNorm2d(512, momentum=momentum)     
        self.relu4 = nn.LeakyReLU(relu_slope) 
        self.dropout4 = nn.Dropout(dropout)

        # 5th block?
        #self.conv2d_5 = nn.Conv2D(512, 1024, kernel_size=4, stride=2, padding=1, bias=False)
        #self.batch_norm5 = nn.BatchNorm2d(1024, momentum=momentum)      
        #self.relu5 = nn.LeakyReLU(relu_slope) 
        #self.dropout5 = nn.Dropout(dropout)

        self.conv2d_final = nn.Conv2d(512, 1, kernel_size=1, stride=1, padding=0, bias = False)
        # activation sigmoid
        #self.flatten = layers.Flatten()


    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.conv2d_2(x)
        x = self.batch_norm2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.conv2d_3(x)
        x = self.batch_norm3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.conv2d_4(x)
        x = self.batch_norm4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        #x = self.conv2d_5(x)
        #x = self.batch_norm5(x)
        #x = self.relu5(x)
        #x = self.dropout5(x)
        
        return self.conv2d_final(x).view(x.shape[0], -1)

# Utility Functions

In [154]:
# Gradient penalty is used so that the discriminator loss won't deviate that much.
# This enforces consistent training, so that the discriminator won't learn to much or
# too little.

def gradient_penalty(discriminator, real, fake, device="cpu"):
    cur_batch_size, chan, height, width = real.shape
    beta = torch.rand((cur_batch_size, 1, 1, 1)).repeat(1, chan, height, width).to(device)
    
    # missized tensors
    if(cur_batch_size != batch_size):
        fake = fake[:cur_batch_size]

    interpolated_images = real * beta + fake.detach() * (1 - beta)
    interpolated_images.requires_grad_(True)

    # Calculate discriminator scores
    mixed_scores = discriminator(interpolated_images)

    # Get gradient for the scores (graph needed here because of gradient penalty)
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty


def save_checkpoint(model, optimizer, filename="checkpoint.pth.tar"):
    print("Saving Checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict()
    }
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("Loading Checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location="cuda")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # learning rate saved just in case training afterwards
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def generate_examples(gen, n=100):
    gen.eval()
    for i in range(n):
        with torch.no_grad():
            noise = torch.randn(1, noise_dim, 1, 1).to(device)
            img = gen(noise)
            save_image(img * 0.5 + 0.5, f'basicgan_examples/img_{i}.png')

    gen.train()


# Training can happen now

In [155]:
# Will hopefully speedup since I'm using images of the same resolution for training
torch.backends.cudnn.benchmarks = True

# Custom batch loader since I am using variable batch sizes at each step
def get_loader(image_size):
    # image-readable format
    transform = transforms.Compose(
        [
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Normalize(
                [0.5 for _ in range(channels)],
                [0.5 for _ in range(channels)]
            )
        ]
    )
    dataset = datasets.ImageFolder(root=datapath, transform=transform)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    return loader, dataset


# Training loop
def train_loop(dis, gen, loader, dataset, opt_dis, opt_gen, scaler_gen, scaler_dis, do_grad_pen=True):
    loop = tqdm(loader, leave=True)
    for batch_index, (real, _) in enumerate(loop):
        real = real.to(device)

        # Train Discriminator
        noise = torch.randn(batch_size, noise_dim, 1, 1).to(device)

        with torch.amp.autocast("cuda"):
            fake = gen(noise)
            dis_real = dis(real)
            dis_fake = dis(fake.detach())
            gp = gradient_penalty(dis, real, fake, device)
            if(do_grad_pen == False):
                gp = 0
            # minimize mistakes
            loss_dis = (
                -(torch.mean(dis_real) - torch.mean(dis_fake))
                + 10 * gp
                + (0.001 * torch.mean(dis_real ** 2))
            )

        opt_dis.zero_grad()
        scaler_dis.scale(loss_dis).backward()
        scaler_dis.step(opt_dis)
        scaler_dis.update()

        # Train Generator:
        with torch.amp.autocast("cuda"):
            gen_fake = dis(fake)
            loss_gen = -torch.mean(gen_fake)

        opt_gen.zero_grad()
        scaler_gen.scale(loss_gen).backward()
        scaler_gen.step(opt_gen)
        scaler_gen.update()

        # show losses in tqdm
        if(do_grad_pen==True):
            loop.set_postfix(
                gp=gp.item(),
                loss_dis=loss_dis.item()
            )
        else:
            loop.set_postfix(
                loss_dis=loss_dis.item()
            )

In [156]:
# models
gen = Generator(noise_dim, img_size, channels).to(device)
dis = Discriminator(img_size, channels).to(device)

# optimizers
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.0, 0.99))
opt_dis = optim.Adam(dis.parameters(), lr=lr, betas=(0.0, 0.99))

scaler_dis = torch.amp.GradScaler("cuda")
scaler_gen = torch.amp.GradScaler("cuda")

# load model if retraining
if(load_model):
    load_checkpoint(gen_save_path, gen, opt_gen, lr)
    load_checkpoint(dis_save_path, dis, opt_dis, lr)


if(load_model == False): # Train
    gen.train()
    dis.train()

    loader, dataset = get_loader(img_size)

    for epoch in range(total_epochs):
        print(f"Epoch [{epoch+1}/{total_epochs}]")
        train_loop(dis, gen, loader, dataset, opt_dis, opt_gen, scaler_gen, scaler_dis, grad_pen)

        if(save_model==True):
            save_checkpoint(gen, opt_gen, filename=gen_save_path)
            save_checkpoint(dis, opt_dis, filename=dis_save_path)
            generate_examples(gen, 20)

Epoch [1/250]


100%|██████████| 467/467 [00:24<00:00, 19.21it/s, gp=0.0517, loss_dis=0.563]  


Saving Checkpoint
Saving Checkpoint
Epoch [2/250]


100%|██████████| 467/467 [00:23<00:00, 20.01it/s, gp=0.00969, loss_dis=0.145] 


Saving Checkpoint
Saving Checkpoint
Epoch [3/250]


...

Saving Checkpoint
Saving Checkpoint
Epoch [249/250]


100%|██████████| 467/467 [00:24<00:00, 19.15it/s, gp=0.00629, loss_dis=-0.0571]   


Saving Checkpoint
Saving Checkpoint
Epoch [250/250]


100%|██████████| 467/467 [00:25<00:00, 18.65it/s, gp=0.00343, loss_dis=0.0167]   


Saving Checkpoint
Saving Checkpoint


In [157]:
# Display for presentation purposes
def display(
    images, n=10, size=(20, 3), cmap="gray_r", as_type="float32", save_to=None
):
    """
    Displays n random images from each one of the supplied arrays.
    """

    plt.figure(figsize=size)
    for i in range(n):
        _ = plt.subplot(1, n, i + 1)
        plt.imshow((images[i].astype(as_type) + 1) / 2, cmap=cmap)
        plt.axis("off")

    if save_to:
        plt.savefig(save_to)
        print(f"\nSaved to {save_to}")

    plt.show()


def preprocess(img):
    img = (img.astype(np.float32) - 127.5)/ 127.5
    return img

In [158]:
if(load_model):
    # Evaluate
    generate_examples(gen, 10)

    # get directory
    cwd = os.getcwd()
    path = cwd + "/basicgan_examples"
    images = []

    # get all images
    for image in os.listdir(path):
        image_path = os.path.join(path, image)
        img = cv2.imread(image_path)
        if(image is not None):
            images.append(img)
    
    train = list(map(preprocess, images))

    display(train[:10])